# Inversion

We benchmark the following models to the inverse heat conduction problem:

1. Finite difference - difference between two forward models
2. Tangent linear - derivative of the problem in forward mode
3. Adjoint model - gradient of the objective function w.r.t. inversion variables

In [ ]:
import numpy as np
from conduction import ConductionND
from conduction import Inversion
from petsc4py import PETSc
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
minX, maxX = 0.0, 1000.0
minY, maxY = 0.0, 1000.0
minZ, maxZ = -35e3, 1000.0
nx, ny, nz = 10, 9, 10
n = nx*ny*nz

mesh = ConductionND((minX, minY, minZ), (maxX, maxY, maxZ), (nx,ny,nz))

# BCs
mesh.boundary_condition('maxZ', 298.0, flux=False)
mesh.boundary_condition('minZ', 0.04, flux=True)

In [ ]:
lithology = np.zeros((nz,ny,nx), dtype='int32')
lithology[:,3:7,:] = 1
lithology[:,7:,:]  = 2

In [ ]:
plt.pcolor(lithology[5,:,:])
plt.colorbar()

In [ ]:
inv = Inversion(lithology.flatten(), mesh)

## Construct the inverse problem

We take bits of the `Inversion` module and tailor the forward and inverse problem.

In [ ]:
def nonLinearConductivity(k0, T, a):
    return k0*(298.0/T)**a

def objective_function(self, x, x0, sigma_x0):
    return np.sum((x - x0)**2/sigma_x0**2)

def forward_model(inv, X):
    k_list, H_list, a_list = np.array_split(X.array[:-1], 3)
    q0 = X.array[-1]
    
    k0, H, a = inv.map(k_list, H_list, a_list)
    
    inv.mesh.update_properties(k0, H)
    inv.mesh.boundary_condition('maxZ', 298.0, flux=False)
    inv.mesh.boundary_condition('minZ', q0, flux=True)
    
    T = inv.linear_solve()
    
    

In [ ]:
k = np.array([3.5, 2.0, 3.2])
H = np.array([0.0, 1e-6, 2e-6])
a = np.array([0., 0., 0.])
q0 = 35e-3

# Inversion variables
x = PETSc.Vec().create()
x.setSizes(k.size*3+1)
x.setUp()
x.setArray(np.hstack([k, H, a, [q0]]))

# Priors
k_prior = k*1.1
H_prior = H*1.1
a_prior = a*1.1

sigma_k = k*0.1
sigma_H = H*0.1
sigma_a = a*0.1

inv.add_prior(k=(k_prior,sigma_k), H=(H_prior,sigma_H), a=(a_prior,sigma_a), q0=(30e-3, 5e-3))

cost = inv.forward_model(x)
print cost

In [ ]:
T = inv.temperature.array.reshape(nz,ny,nx)
plt.imshow(T[:,:,5]-273.14, origin='lower')
plt.colorbar()

In [ ]:
dx = x*0.01
gradient = x.duplicate()

fm0 = inv.forward_model(x)
fm1 = inv.forward_model(x+dx)

tl = inv.tangent_linear(x, dx)

ad = inv.adjoint(None, x, gradient)

print "finite difference", (fm1 - fm0)
print "tangent linear", tl[1]
print "adjoint", gradient.array.dot(dx)

In [ ]:
inv = Inversion(lithology.flatten(), mesh)

np.random.seed(0)

q_obs = np.ones(5)*0.03
sigma_q = q_obs*0.5
q_coord = np.zeros((5,3))
q_coord[:,0] = np.random.random(5)*1e3
q_coord[:,1] = 0.0
q_coord[:,2] = np.random.random(5)*1e3

inv.add_observation(q=(q_obs, sigma_q, q_coord))

In [ ]:
dx = x*0.001

fm0 = inv.forward_model(x)
fm1 = inv.forward_model(x+dx)
tl = inv.tangent_linear(x,dx)
ad = inv.adjoint(None, x, gradient)

print "finite difference", (fm1 - fm0)
print "tangent linear", tl[1]
print "adjoint", gradient.array.dot(dx)

In [ ]:
inv = Inversion(lithology.flatten(), mesh)

T_prior = np.ones(n)*400.
sigma_T = T_prior*0.01

inv.add_prior(T=(T_prior,sigma_T))

In [ ]:
dx = x*0.01

fm0 = inv.forward_model(x)
fm1 = inv.forward_model(x+dx)
tl = inv.tangent_linear(x,dx)
ad = inv.adjoint(None, x, gradient)

print "finite difference", (fm1 - fm0)
print "tangent linear", tl[1]
print "adjoint", gradient.array.dot(dx)

In [ ]:
inv.ksp.setDMActive(True)
inv.ksp.setDM(inv.mesh.dm)
inv.ksp.setComputeOperators(inv.mesh.mat)

Compare to 2D example

In [ ]:
from benpy import Inversion as Inversion2
from scipy.sparse import coo_matrix
from scipy.sparse.linalg import spsolve

Xcoords = np.linspace(minX, maxX, nx)
Ycoords = np.linspace(minY, maxY, ny)
xq, yq = np.meshgrid(Xcoords,Ycoords)

inv2 = Inversion2(xq.ravel(), yq.ravel(), lithology[5,:,:].flatten())

In [ ]:
inv2.add_prior(Qs=(q_obs, sigma_q, q_coord[:,0]))
fm0 = inv2.forward_model(x)
fm1 = inv2.forward_model(x+dx)
tl = inv2.tangent_linear(x,dx)
ad = inv2.adjoint(x)

print "finite difference", (fm1 - fm0)
print "tangent linear", tl[1]
print "adjoint", ad[1].dot(dx)
print ad

In [ ]:
inv2 = Inversion2(xq.ravel(), yq.ravel(), lithology[5,:,:].flatten())
T_prior = np.ones(nx*ny)*400.
sigma_T = T_prior*0.01
inv2.add_prior(T=(T_prior, sigma_T))


fm0 = inv2.forward_model(x)
fm1 = inv2.forward_model(x+dx)
tl = inv2.tangent_linear(x,dx)
ad = inv2.adjoint(x)

print "finite difference", (fm1 - fm0)
print "tangent linear", tl[1]
print "adjoint", ad[1].dot(dx)
print ad

## TAO solve

Solve a minimisation problem. TAO provides a number of minimisation schemes, some which require the gradient and Hessian.

In [ ]:
from petsc4py import PETSc
from time import clock

In [ ]:
# x as a PETSc vector
inv_x = PETSc.Vec().create()
inv_x.setSizes(x.size)
inv_x.setUp()
inv_x.setArray(x)

lower_bound = inv_x.duplicate()
lower_bound.array[:3] = 0.5 # conductivity
lower_bound.array[-1] = 5e-3 # q0

upper_bound = inv_x.duplicate()
upper_bound.array[:3] = 4.5 #conductivity
upper_bound.array[3:6] = 5e-6 # heat production
upper_bound.array[6:9] = 1.0 # a
upper_bound.array[-1] = 45e-3 # q0


tao = PETSc.TAO().create()
tao.setType('blmvm')
# tao.setType(PETSc.TAO.Type.POUNDERS)
tao.setVariableBounds(lower_bound, upper_bound)
tao.setObjectiveGradient(inv.adjoint)
tao.setFromOptions()
# ksp = tao.getKSP()
# ksp.setType('gmres')
# ksp.setOperators(inv.mesh.mat)
# pc = ksp.getPC()
# pc.setType('gamg')
# ksp.setTolerances(1e-8,1e-8)
# ksp.setFromOptions()
# inv.ksp = ksp

t = clock()
tao.solve(inv_x)
print "Completed in", clock()-t
print tao.getConvergedReason(), tao.getIterationNumber()

In [ ]:
from scipy.optimize import minimize

def adjoint(x):
    """ need this handler for PETSc vector convention """
    inv_x.setArray(x)
    c = inv.adjoint(None, inv_x, gradient)
    return c, gradient.array
    

t = clock()
opt = minimize(adjoint, x.array, method='TNC', jac=True, bounds=zip(lower_bound.array, upper_bound.array))
print clock()-t

In [ ]:
import pyOpt

opt = pyOpt.Optimization('No problem', adjoint)
opt.addObj('f')
for i in range(x.array.size):
    opt.addVar('v%i'%i, 'c', lower=lower_bound.array[i], upper=upper_bound.array[i], value=x.array[i])
print opt

In [ ]:
solver = pyOpt.SLSQP()
# solver.setOption('IPRINT')
solver(opt, sens_type='CS', sens_mode='pgc')
print opt.solution

In [ ]:
print tao.getConvergedReason(), tao.getIterationNumber()

In [ ]:
dAdkl2 = coo_matrix((inv2.Vsp[0], (inv2.Isp[0], inv2.Jsp[0])), shape=(inv2.Nnode,inv2.Nnode))

In [ ]:
df = 1.0

obs = inv.observation['q']

# Compute heat flux
gradTz, gradTy, gradTx = np.gradient(T.reshape(inv.mesh.nz, inv.mesh.ny, inv.mesh.nx),
                                     inv.mesh.dx, inv.mesh.dy, inv.mesh.dz)
heatflux = inv.mesh.diffusivity.reshape(inv.mesh.nz, inv.mesh.ny, inv.mesh.nx)*(gradTz + gradTy + gradTx)

inv.interp.values = heatflux
q_interp = inv.interp(obs[2])
cost += inv.objective_function(q_interp, obs[0], obs[1])

## AD
dcdq = inv.objective_function_ad(q_interp, obs[0], obs[1])
dq_interp_ad = dcdq*df
dq_ad = inv.interp.adjoint(obs[2], dq_interp_ad)
dqdz = inv.mesh.diffusivity.reshape(inv.mesh.nz, inv.mesh.ny, inv.mesh.nx)
dqdy = inv.mesh.diffusivity.reshape(inv.mesh.nz, inv.mesh.ny, inv.mesh.nx)
dqdx = inv.mesh.diffusivity.reshape(inv.mesh.nz, inv.mesh.ny, inv.mesh.nx)
dqdk = gradTz + gradTy + gradTx

dk_ad = (dqdk*dq_ad)

dz_ad = dqdz*dq_ad
dy_ad = dqdy*dq_ad
dx_ad = dqdx*dq_ad

dqdTz = inv.mesh.diffusivity.reshape(inv.mesh.nz, inv.mesh.ny, inv.mesh.nx)/inv.mesh.dz
dqdTy = inv.mesh.diffusivity.reshape(inv.mesh.nz, inv.mesh.ny, inv.mesh.nx)/inv.mesh.dy
dqdTx = inv.mesh.diffusivity.reshape(inv.mesh.nz, inv.mesh.ny, inv.mesh.nx)/inv.mesh.dz

dT_ad = (dqdTx*dq_ad + dqdTy*dq_ad + dqdTz*dq_ad)

In [ ]:
plt.imshow(dk_ad[:,-1,:], origin='lower', interpolation='nearest')